In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score 

In [5]:
digits=load_digits()
X,y=digits.data, digits.target 

In [6]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2,random_state=11)

In [9]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, target):
        self.data=torch.tensor(data, dtype=torch.float32)
        self.target=torch.tensor(target, dtype=torch.long)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        sample={'data':self.data[index], 'target':self.target[index]}
        return sample 

In [11]:
train_dataset=CustomDataset(X_train, y_train)
test_dataset=CustomDataset(X_test, y_test)

In [14]:
train_dataset[1]

{'data': tensor([ 0.0000, -0.3266, -0.2477,  0.9870, -1.3667, -1.0131, -0.4107, -0.1187,
         -0.0558, -0.6240,  0.2959,  0.7518, -1.9585, -1.3309, -0.5054, -0.1243,
         -0.0354, -0.4485,  1.0672,  0.6793, -1.1479, -1.2392, -0.5341, -0.1049,
         -0.0373,  1.7479,  1.1191, -1.0058, -1.6104, -0.0810,  0.4738, -0.0264,
          0.0000,  2.7501,  1.3089, -0.5060,  0.1192,  1.2237,  1.1576,  0.0000,
         -0.0577,  1.8022,  1.3734,  1.3643,  1.1887, -0.9234, -0.8084, -0.0940,
         -0.0334, -0.4056, -0.6385,  1.2375,  0.1230, -1.4744, -0.7676, -0.2048,
         -0.0264, -0.2901, -0.2996,  0.8951, -1.1972, -1.1653, -0.4980, -0.1942]),
 'target': tensor(4)}

In [ ]:
train_dataset